In [1]:
import pandas as pd
import numpy as np
from os.path import expanduser
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
temp = {
    'system' :[],
    'cpu': [],
    'steps': []
}
home = expanduser("~")
df = pd.read_csv('./model-results-2x-not-that-late-1x-33p.csv')

for system_id in range(300):
    system_id += 1
    system_id = str(system_id)
    
    core_count = df[df.approach == 'BNV2-4.0'][df.system == 'system_' + system_id].max_total_core.values[0] * 2
    n_iter = df[df.approach == 'BNV2-4.0'][df.system == 'system_' + system_id].steps.values[0]
    
    if not os.path.exists('./results/'+system_id+'.txt'): continue
    
    with open('./results/'+system_id+'.txt') as f:
        lines = f.read().split('\n')
        lines = [line for line in lines if len(line.strip()) > 0]
        lines = [line for line in lines if 'info' not in line]
    
    initial_samples = 5
    if n_iter + initial_samples > len(lines):
        continue
    temp['system'].append('system_' + system_id)
#     print(system_id)
    cpu = 1e10
    step = 1e10
    for iter_id, line in enumerate(lines):
        meets = line.split(' ')[1] == 'True'
        total = np.round(float(line.split(' ')[2]),2)
#         print(meets, total)
        if meets == False: 
            continue
        
        if cpu > total:
#             print('updated', cpu, step)
            cpu = total
            step = iter_id + 1
#     print()
    temp['cpu'].append(cpu)
    temp['steps'].append(step)
df_mobo = pd.DataFrame(temp)
df_mobo

,system,cpu,steps
0,system_1,141.44,3.0
1,system_2,224.19,3.0
2,system_3,162.13,3.0
3,system_4,162.13,3.0
4,system_5,127.65,3.0
...,...,...,...
239,system_296,141.44,3.0
240,system_297,162.13,3.0
241,system_298,141.44,3.0
242,system_299,148.13,4.0


In [4]:
'didnt find anything valid', df_mobo[df_mobo.cpu > 1e9].shape[0]

('didnt find anything valid', 92)

In [5]:
df_mobo = df_mobo[df_mobo.cpu < 1e9]

In [6]:
df_bnv2_4 = df[df.approach == 'BNV2-4.0']

In [7]:
df_bnv2_4.head()

,approach,system,steps,min_total_core,max_total_core,resources,classes,sla
5,BNV2-4.0,system_1,8,70.8,82.0,10,12,300.0
13,BNV2-4.0,system_2,26,116.5,130.0,10,12,300.0
21,BNV2-4.0,system_3,14,85.0,94.0,10,12,300.0
29,BNV2-4.0,system_4,12,87.0,94.0,10,12,300.0
37,BNV2-4.0,system_5,10,58.8,74.0,10,12,300.0


In [8]:
df_mobo.head()

,system,cpu,steps
0,system_1,141.44,3.0
1,system_2,224.19,3.0
2,system_3,162.13,3.0
3,system_4,162.13,3.0
4,system_5,127.65,3.0


In [9]:
mobo_is_cheaper = []
diffs = []
systems = df_mobo.system.values
for system in systems:
    mobo_cpu = df_mobo[df_mobo.system == system].cpu.values[0]
    bnv2_4_cpu = df_bnv2_4[df_bnv2_4.system == system].max_total_core.values[0]
    if mobo_cpu < bnv2_4_cpu:
        mobo_is_cheaper.append(system)
    diffs.append((mobo_cpu - bnv2_4_cpu) / (bnv2_4_cpu))
print(len(mobo_is_cheaper),'caces mobo is better', 'total is ', len(systems))
print('on average MOBO uses',str(int(np.round(np.mean(diffs) * 100))) + '%','more CPU shares to meet the SLAs')

0 caces mobo is better total is  152
on average MOBO uses 74% more CPU shares to meet the SLAs
